<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Mean-Variance Portfolio Theory

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Theory

Topics of interest include:

* possible risk-return combinations
* optimal portfolios
* capital market line (CML)

## Real Financial Data

**_Historical end-of-day financial time series data._**

See Python for Finance (2nd ed., ch. 13)  and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Portfolio Optimization

### The Data

In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
from pylab import mpl, plt

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
plt.style.use('seaborn-v0_8')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                 index_col=0, parse_dates=True).dropna()

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'SPY', 'GLD']  

In [ ]:
noa = len(symbols)  

In [ ]:
data = raw[symbols]

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
rets.hist(bins=40, figsize=(10, 8));

In [ ]:
rets.mean() * 252

In [ ]:
rets.cov() * 252  

### The Basic Theory

In [ ]:
weights = np.random.random(noa)  
weights /= np.sum(weights)

In [ ]:
weights

In [ ]:
weights.sum()

In [ ]:
np.sum(rets.mean() * weights) * 252  

In [ ]:
np.dot(weights.T, np.dot(rets.cov() * 252, weights))  

In [ ]:
math.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))  

In [ ]:
def port_ret(weights):
    return np.sum(rets.mean() * weights) * 252

In [ ]:
def port_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))

In [ ]:
prets = []
pvols = []
for p in range (2500):  
    weights = np.random.random(noa)  
    weights /= np.sum(weights)  
    prets.append(port_ret(weights))  
    pvols.append(port_vol(weights))  
prets = np.array(prets)
pvols = np.array(pvols)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(pvols, prets, c=prets / pvols,
            marker='o', cmap='coolwarm')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio');

### Portfolio Optimizations

In [ ]:
import scipy.optimize as sco

In [ ]:
def min_func_sharpe(weights):  
    return -port_ret(weights) / port_vol(weights)  

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})  

In [ ]:
bnds = tuple((0, 1) for x in range(noa))  

In [ ]:
eweights = np.array(noa * [1. / noa,])  
eweights  

In [ ]:
min_func_sharpe(eweights)

In [ ]:
# sco.minimize?

In [ ]:
%%time
opts = sco.minimize(min_func_sharpe, eweights,
                    method='SLSQP', bounds=bnds,
                    constraints=cons)  

In [ ]:
opts  

In [ ]:
opts['x'].round(3)  

In [ ]:
port_ret(opts['x']).round(3)  

In [ ]:
port_vol(opts['x']).round(3)  

In [ ]:
port_ret(opts['x']) / port_vol(opts['x'])  

In [ ]:
optv = sco.minimize(port_vol, eweights,
                    method='SLSQP', bounds=bnds,
                    constraints=cons)  

In [ ]:
optv

In [ ]:
optv['x'].round(3)

In [ ]:
port_vol(optv['x']).round(3)

In [ ]:
port_ret(optv['x']).round(3)

In [ ]:
port_ret(optv['x']) / port_vol(optv['x'])

### Efficient Frontier

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x:  port_ret(x) - tret},
        {'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})  

In [ ]:
bnds = tuple((0, 1) for x in weights)

In [ ]:
%%time
trets = np.linspace(0.05, 0.2, 50)
tvols = []
for tret in trets:
    res = sco.minimize(port_vol, eweights, method='SLSQP',
                       bounds=bnds, constraints=cons)  
    tvols.append(res['fun'])
tvols = np.array(tvols)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(pvols, prets, c=prets / pvols,
            marker='.', alpha=0.8, cmap='coolwarm')
plt.plot(tvols, trets, 'b', lw=4.0)
plt.plot(port_vol(opts['x']), port_ret(opts['x']),
         'y*', markersize=15.0)
plt.plot(port_vol(optv['x']), port_ret(optv['x']),
         'r*', markersize=15.0)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio');

### Capital Market Line

In [ ]:
import scipy.interpolate as sci

In [ ]:
ind = np.argmin(tvols)  # pick index of minimum vol portfolio
evols = tvols[ind:]  
erets = trets[ind:]  

In [ ]:
tck = sci.splrep(evols, erets)  

In [ ]:
def f(x):
    ''' Efficient frontier function (splines approximation). '''
    return sci.splev(x, tck, der=0)
def df(x):
    ''' First derivative of efficient frontier function. '''
    return sci.splev(x, tck, der=1)

In [ ]:
def equations(p, rf=0.01):
    eq1 = rf - p[0]  
    eq2 = rf + p[1] * p[2] - f(p[2])  
    eq3 = p[1] - df(p[2])  
    return eq1, eq2, eq3

In [ ]:
opt = sco.fsolve(equations, [0.01, 0.5, 0.15])  

In [ ]:
opt  

In [ ]:
np.round(equations(opt), 6)  

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(pvols, prets, c=(prets - 0.01) / pvols,
            marker='.', cmap='coolwarm')
plt.plot(evols, erets, 'b', lw=4.0)
cx = np.linspace(0.0, 0.3)
plt.plot(cx, opt[0] + opt[1] * cx, 'r', lw=1.5)
plt.plot(opt[2], f(opt[2]), 'y*', markersize=15.0) 
plt.grid(True)
plt.axhline(0, color='k', ls='--', lw=2.0)
plt.axvline(0, color='k', ls='--', lw=2.0)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio');

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x:  port_ret(x) - f(opt[2])},
        {'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})  
res = sco.minimize(port_vol, eweights, method='SLSQP',
                   bounds=bnds, constraints=cons)

In [ ]:
res['x'].round(3)  

In [ ]:
port_ret(res['x'])

In [ ]:
port_vol(res['x'])

In [ ]:
port_ret(res['x']) / port_vol(res['x'])

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>